Extracting IQAir API 

In [4]:
import requests
url = "https://api.airvisual.com/v2/city"
params = {
    "city": "Delhi",
    "state": "Delhi",
    "country": "India",
    "key": "86a4d7ad-f925-44cc-a3a9-21e07f3f3041"
}
data = requests.get(url, params=params).json()
print(data)

{'status': 'success', 'data': {'city': 'Delhi', 'state': 'Delhi', 'country': 'India', 'location': {'type': 'Point', 'coordinates': [77.1167, 28.6683]}, 'current': {'pollution': {'ts': '2025-06-08T10:00:00.000Z', 'aqius': 152, 'mainus': 'p2', 'aqicn': 120, 'maincn': 'p1'}, 'weather': {'ts': '2025-06-08T10:00:00.000Z', 'ic': '50d', 'hu': 20, 'pr': 1001, 'tp': 43, 'wd': 320, 'ws': 4.12}}}}


Cleaning and transform IQAir API 

In [3]:

if data["status"] == "success":
    pollution = data['data']['current']['pollution']
    weather = data['data']['current']['weather']
    coords = data['data']['location']['coordinates']

    transformed = {
        "city": data["data"]["city"],
        "state": data["data"]["state"],
        "country": data["data"]["country"],
        "aqi": pollution["aqius"],
        "pm_source": pollution["mainus"],
        "timestamp": pollution["ts"],
        "temperature": weather["tp"],
        "humidity": weather["hu"],
        "latitude": coords[1],
        "longitude": coords[0]
    }

    print("✅ Cleaned data:")
    print(transformed)

else:
    print("❌ Failed to fetch data:", data["data"]["message"])


✅ Cleaned data:
{'city': 'Delhi', 'state': 'Delhi', 'country': 'India', 'aqi': 151, 'pm_source': 'p2', 'timestamp': '2025-06-08T08:00:00.000Z', 'temperature': 41, 'humidity': 24, 'latitude': 28.6683, 'longitude': 77.1167}
